In [1]:
import pandas as pd
import numpy as np

# Load the CSV
df = pd.read_csv('/home/texturelab/Desktop/mrinmoy projects/bearing_steel_carbide_predictions.csv')
df=pd.re
# Quick peek
print(df.shape)
print(df.head())


(4730, 8)
   Austenitization_Temp_C  Quenching_Temp_C  Carbon_Content_%  \
0              862.417854        182.165549          0.934687   
1              846.543392        199.251798          1.093774   
2              866.192213        184.308184          1.087950   
3              888.075746        207.101111          0.898741   
4              844.146166        204.364210          1.065777   

   Chromium_Content_%  Holding_Time_min  Cooling_Rate_C_per_s Carbide_Banding  \
0            1.549915         27.403043             22.642021            High   
1            1.461287         38.159444             24.673078        Moderate   
2            1.455058         31.481477             13.084678            High   
3            1.571525         30.473692             18.656031             Low   
4            1.536980         31.941371             10.080195             Low   

  Interface_Stability  
0              Stable  
1            Unstable  
2            Unstable  
3              S

In [2]:
df.head()

,Austenitization_Temp_C,Quenching_Temp_C,Carbon_Content_%,Chromium_Content_%,Holding_Time_min,Cooling_Rate_C_per_s,Carbide_Banding,Interface_Stability
0,862.417854,182.165549,0.934687,1.549915,27.403043,22.642021,High,Stable
1,846.543392,199.251798,1.093774,1.461287,38.159444,24.673078,Moderate,Unstable
2,866.192213,184.308184,1.087950,1.455058,31.481477,13.084678,High,Unstable
3,888.075746,207.101111,0.898741,1.571525,30.473692,18.656031,Low,Stable
4,844.146166,204.364210,1.065777,1.536980,31.941371,10.080195,Low,Unstable


In [5]:
X = df.drop(['Carbide_Banding', 'Interface_Stability'], axis=1)
y_int  = df['Interface_Stability']
y_band=df['Carbide_Banding']


In [7]:
from sklearn.preprocessing import LabelEncoder
le_band = LabelEncoder()
yb = le_band.fit_transform(y_band)

In [8]:

le_int = LabelEncoder()
yi = le_int.fit_transform(y_int)

In [9]:
print(dict(zip(le_band.classes_, le_band.transform(le_band.classes_))))
print(dict(zip(le_int.classes_,  le_int.transform(le_int.classes_))))

{'High': 0, 'Low': 1, 'Moderate': 2}
{'Stable': 0, 'Unstable': 1}


In [10]:
from sklearn.model_selection import train_test_split

# Stratify on carbide banding to preserve class balance
X_train, X_test, yb_train, yb_test, yi_train, yi_test = train_test_split(
    X, yb, yi,
    test_size=0.20,
    random_state=42,
    stratify=yb
)

print("Train size:", X_train.shape, "Test size:", X_test.shape)


Train size: (3784, 6) Test size: (946, 6)


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Identify numeric columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Impute → scale
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',  StandardScaler())
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols)
])


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 5a. Carbide Banding (3-class)
band_pipeline = Pipeline([
    ('preproc', preprocessor),
    ('clf',     RandomForestClassifier(
                    n_estimators=100,
                    random_state=42
                ))
])
band_pipeline.fit(X_train, yb_train)
yb_pred = band_pipeline.predict(X_test)

print("=== Carbide Banding ===")
print(classification_report(yb_test, yb_pred,
      target_names=le_band.classes_))

# 5b. Interface Stability (binary)
int_pipeline = Pipeline([
    ('preproc', preprocessor),
    ('clf',     RandomForestClassifier(
                    n_estimators=100,
                    random_state=42
                ))
])
int_pipeline.fit(X_train, yi_train)
yi_pred = int_pipeline.predict(X_test)

print("=== Interface Stability ===")
print(classification_report(yi_test, yi_pred,
      target_names=le_int.classes_))


=== Carbide Banding ===
              precision    recall  f1-score   support

        High       0.16      0.05      0.08       224
         Low       0.40      0.58      0.47       387
    Moderate       0.35      0.34      0.34       335

    accuracy                           0.37       946
   macro avg       0.31      0.32      0.30       946
weighted avg       0.33      0.37      0.33       946

=== Interface Stability ===
              precision    recall  f1-score   support

      Stable       0.60      0.86      0.70       558
    Unstable       0.45      0.17      0.24       388

    accuracy                           0.57       946
   macro avg       0.52      0.51      0.47       946
weighted avg       0.54      0.57      0.51       946

